In [ ]:
import pandas as pd
d = pd.read_excel("EUR/Energy/Polak-Kwartier-20190101.xlsx", names=["time", "kWh"], converters= {"Datum-tijd tot": pd.to_datetime}, skiprows=4, index_col=0)
d

In [ ]:
# What effect do vars have on co2with 
import os
def read_energy_files(dir):                                                                                                  
    r = []
    for root, dirs, files in os.walk(dir):
        if dirs and not dirs[0].isdigit():
            continue    
        for year_dir in dirs:
            building_usages = os.path.join(root, os.path.join(year_dir,"EAN 26728"))
            for year_root, year_dirs, year_files in os.walk(building_usages):
                for building_dir in year_dirs:
                    for building_root, building_dirs, building_files in os.walk(os.path.join(year_root, building_dir)):
                        print("dir:",building_dir, "files", building_files)
                    
                        for name in building_files[-1:]: 
                            d = pd.read_excel(os.path.join(building_root, name), names=["time", "kWh"], converters= {"Datum-tijd tot": pd.to_datetime}, skiprows=4, index_col=0)
                            d["building_name"] = building_dir
                            r.append(d)
    return r 
energy_usages = pd.concat(read_energy_files(r"EUR\Energy"))
# energy_usages.to_csv("EUR/Integrated/energy_usages.csv")

In [ ]:
d["month"] = pd.DatetimeIndex(d.index).month 
d["year"] = pd.DatetimeIndex(d.index).year
d = d[d["year"]==2019]
d.tail(4)

monthly_means = d.groupby(pd.Grouper(level='time', freq='D'))["kWh"].sum()
monthly_means = monthly_means.groupby(pd.Grouper(level='time', freq='M')).sum()
# daily_mean_for_year = monthly_means.mean()
print(monthly_means)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig = plt.figure(figsize=(32, 12))
# fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(8,8))
# axs= axs.ravel()
# month_grps = d.groupby("month")

# for ax, (i, sub) in zip(axs, month_grps):
#     sub["kWh"].plot(kind="line", x="time", legend=True, title=i, ax=ax)
# monthly_means = d.resample("M").mean()with 
# print(monthly_means)
monthly_means = d.groupby(pd.Grouper(level='time', freq='D'))["kWh"].sum()
monthly_means = monthly_means.groupby(pd.Grouper(level='time', freq='M')).mean()
daily_mean_for_year = monthly_means.mean()
print(daily_mean_for_year)
# print(monthly_means)

for i in range(1, (12+1)):
    ax = fig.add_subplot(4, 3, i)
    # print(f"len is {len(d)}")
    plot_d = d[d["month"]==i]
    # monthly_mean = monthly_means[monthly_means["month"]==i]
    monthly_mean = monthly_means[monthly_means.index.month==i]
    mean = monthly_mean.iloc[0]
    # print(mean)
    # print(i, monthly_mean)
    # print(i, monthly_mean.head(1))
    # print(f"len is {len(plot_d)}")
    # print(plot_d.tail(2))
    # day_grps = plot_d.groupby(pd.Grouper(level='time', freq='D')).sum()
    day_grps = plot_d.groupby(pd.Grouper(level='time', freq='D'))["kWh"].sum()

    # print(i)
    # print(day_grps)
    day_grps.plot(kind="line", y="kWh", ax=ax)
    # monthly_mean.plot(kind="line", ax=ax)
    ax.axhline(y=mean, color="g", alpha=0.5, linestyle="-")
    ax.axhline(y=daily_mean_for_year, alpha=1, color="y", linestyle="dotted")
    ax.legend(["daily kWh", "local daily avg", "global daily avg"])
    ax.set_xlabel("Day", labelpad=-3)
    ax.set_ylabel("Electricity Usage (kWh)")
    ax.grid()
    # ax.lin
plt.tight_layout()

In [ ]:
# Plotting GBS data 
# https://stackoverflow.com/questions/55776092/plotting-a-graph-by-month-wise-in-pandas
# https://stackoverflow.com/questions/61443949/python-pandas-group-by-then-plot-by-category
import re

gbs_val_descriptions = {
    "01FLOW":"airflow-sensor",
    "01RS-CV":"regelsign-verwarming",
    "01RS-GKW":"regelsign-koeling",
    "01RS-VAV":"regelsign-vav", # Variable air volumewith 
    "01PIR":"pir", # Infrared sensor
    "01RM":"ruimte-modus",
    "01RMT":"co2-ruimte",
    "01SP":"setpoint-co2",
    # OAT
    "TT_01BT": "buiten-temperatuur",
    "MT_01RV": "buiten-rv",
    # LBK
    "VA_01PC": "plant-code",
    # LBK 2 excl
    "PT_01SP": "inblaasdruk-setpoint",
    "PT_01IL": "inblaasdruk",
    "TT_03SPB": "inblaastemperatuur-setpoint-berekend",
    "TT_03IL": "inblaastemperatuur",
    "TT_04RL": "retourlucht-temperatuur",
    "WW_01RS": "warmtewiel-sturing",
    "CV_01RS": "regelafsluiter-verwarmer",
    "CV_02RS": "regelafsluiter-koeler",
}

to_drop = []
# file = "2022w18_GBS_trend_Y03-08.xlsx"
# to_drop = ["Y03VAV0339TE-01BSP.PointValue", 'Y03VAV0339TE-01CPA.PointValue', 'Y03VAV0339TE-01RMT.PointValue', 'Y03VAV0339TE-01SP.PointValue']
# file = "2022w18-GBS_trend_OAT .xlsx"
file = "2022w18-GBS_trend-Y-lbk1.xlsx"

gbs = pd.read_excel(f"EUR/{file}", parse_dates=[["Date", "Time"]], index_col=0, sheet_name=list(range(4)))
gbs = pd.concat(gbs.values())
print(f"Reading in {file}\nColumns: {list(gbs.columns)}\nDropping: {to_drop}")
gbs.drop(to_drop, axis=1, inplace=True)


def extractFeature(id):
    if "_" in id:
        match = re.findall(r'[0-9]([^\d]*?_.*)\..*', id)[0]
    else:
        match = re.findall(r'-(.*)\..*', id)[0]
    feature = gbs_val_descriptions[match]
    return feature

gbs = gbs.rename(columns=extractFeature)
gbs.index.name = "time"

date_range = str(gbs.index.min()) + ' to ' +str(gbs.index.max())
print(f"Range of dates is: {date_range}")
# What is BSP and CPA?
# gbs.drop(["plant-code"], axis=1, inplace=True)
gbs = gbs.replace({',': '.'}, regex=True)
print(gbs.astype(float).describe())
gbs.head(20)

In [ ]:
fig, ax = plt.subplots()

gbs_co2 = gbs[["co2-ruimte", "setpoint-co2", "pir"]]
gbs_co2.plot(ax=ax)

# Why is pir 0 even during spikes?